In [ ]:
!pip install focal_loss transformers

     |████████████████████████████████| 3.8 MB 5.4 MB/s 
     |████████████████████████████████| 462 kB 56.4 MB/s 
     |████████████████████████████████| 596 kB 31.6 MB/s 
     |████████████████████████████████| 6.5 MB 40.8 MB/s 
     |████████████████████████████████| 895 kB 41.5 MB/s 
     |████████████████████████████████| 67 kB 4.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoTokenizer, TFAutoModel, AutoConfig
import re
import io
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score,precision_score,recall_score
from zipfile import ZipFile
from IPython.display import FileLink 
import tensorflow as tf
from focal_loss import SparseCategoricalFocalLoss
#import tensorflow_addons as tfa
np.random.seed(45)
tf.random.set_seed(45)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tf.distribute.cluster_resolver.TPUClusterResolver.connect())

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.121.32.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.121.32.130:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
train_data=pd.read_csv("/content/train.csv")
valid_data=pd.read_csv("/content/valid.csv")

In [ ]:
OFF=train_data.loc[(train_data["OFF_label"]==1)&(train_data["HS_label"]==0)].sample(1100)
NORM=train_data.loc[train_data["OFF_label"]==0].sample(1100)
HS=train_data.loc[train_data["HS_label"]!=0]
train_data=OFF.append(NORM)
train_data=train_data.append(HS)
train_data=train_data.sample(frac=1).reset_index(drop=True)

In [ ]:
def encode_labels(data):
    labels=[]
    for row in data.to_dict(orient="records"):
        if row["HS_label"]!=0:
            labels.append(int(row["HS_label"])+1)
        elif row["OFF_label"]!=0:
            labels.append(int(row["OFF_label"]))
        else:
            labels.append(0)
    text=list(data["tweet_text"].astype("str"))
    return text,np.asarray(labels, dtype='int32')

train_text,train_labels=encode_labels(train_data)
valid_text,valid_labels=encode_labels(valid_data)

In [ ]:
def encode_vlabels(data):
    encoded_OFF=tf.convert_to_tensor(data["OFF_label"].astype("int32"))
    encoded_HS=data["HS_label"].astype("int32")
    text=list(data["tweet_text"].astype("str"))
    return text,encoded_OFF,encoded_HS

valid_text,valid_OFF,valid_HS=encode_vlabels(valid_data)

In [ ]:
def Binary_HS_feature_gen(encoded_HS):
    HS_bn=[]
    for x in encoded_HS:
        HS_bn.append(x!=0)
    return tf.convert_to_tensor(HS_bn)
train_HS_bn=Binary_HS_feature_gen(train_data["HS_label"])
valid_HS_bn=Binary_HS_feature_gen(valid_data["HS_label"])

In [ ]:
def tokenize(sentences):
    tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/MARBERTv2")
    input_ids, input_masks = [],[]
    for sentence in sentences:
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True,max_length=256,truncation=True, padding='max_length',return_attention_mask=True, return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
    return np.asarray(input_ids, dtype='int32'),np.asarray(input_masks, dtype='int32')

train_input_ids,train_input_masks=tokenize(train_text)
valid_input_ids,valid_input_masks=tokenize(valid_text)

Downloading:   0%|          | 0.00/439 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def report_gen(predictions,labels):
    report={
    "F1_macro":f1_score(predictions,labels,average="macro"),
    "Accuracy":accuracy_score(predictions,labels),
    "Precision_macro":precision_score(predictions,labels,average="macro"),
    "Recall_macro":recall_score(predictions,labels,average="macro")
    }
    return report

def eval_taskA(predictions,labels_OFF,return_predictions=False):
    predict_class = np.argmax(predictions, axis=1)
    predict_class = predict_class.tolist()
    predictions_OFF=[]
    for i in predict_class:
        if i!=0:
            predictions_OFF.append(1)
        else:
            predictions_OFF.append(0)
    return predictions_OFF if return_predictions else report_gen(predictions_OFF,labels_OFF)

def eval_taskB(predictions,labels_HS_bn,return_predictions=False):
    predict_class = np.argmax(predictions, axis=1)
    predict_class = predict_class.tolist()
    predictions_HS_bn=[]
    for i in predict_class:
        if i!=0 and i!=1:
            predictions_HS_bn.append(1)
        else:
            predictions_HS_bn.append(0)
    return predictions_HS_bn if return_predictions else report_gen(predictions_HS_bn,labels_HS_bn)


def eval_taskC(predictions,labels_HS,return_predictions=False):
    predict_class = np.argmax(predictions, axis=1)
    predict_class = predict_class.tolist()
    predictions_HS=[]
    for i in predict_class:
        if i!=0 and i!=1:
            predictions_HS.append(i-1)
        else:
            predictions_HS.append(0)
    return predictions_HS if return_predictions else report_gen(predictions_HS,labels_HS)


In [ ]:
def create_model(transformer,conv_units=128,qrnn_units=256,dense_units=64):
    input_ids= tf.keras.layers.Input(shape=(256,), dtype='int32')
    input_masks = tf.keras.layers.Input(shape=(256,), dtype='int32')
    embedding_layer=transformer(input_ids, attention_mask=input_masks)[0]
    
    x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=True, dropout=0.3))(embedding_layer)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=True, dropout=0.3))(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=True, dropout=0.3))(x)
    x = tf.keras.layers.Conv1D(128, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
    avg_pool_x= tf.keras.layers.GlobalAveragePooling1D()(x)
    max_pool_x = tf.keras.layers.GlobalMaxPooling1D()(x)
    x = tf.keras.layers.concatenate([avg_pool_x, max_pool_x])
    output=tf.keras.layers.Dense(8, activation="softmax",name="output")(x)

    model = tf.keras.Model(inputs=[input_ids, input_masks], outputs = [output])
    
    return model

In [ ]:
def gen_weights(labels):
    count={}
    for i in labels:
        if i not in count:
            count[i]=1
        else:
            count[i]+=1 
    weights={}
    for i in range(8):
        weights[i]=len(labels)/(8*count[i])
    return weights
weights=gen_weights(train_labels)

In [ ]:
def build_model():
    with tpu_strategy.scope():
      config =AutoConfig.from_pretrained("UBC-NLP/MARBERTv2",dropout=0.3,seed=3,attention_dropout=0.3,output_hidden_states = True)
      transformer= TFAutoModel.from_pretrained("UBC-NLP/MARBERTv2",config=config)
      return create_model(transformer)

model=build_model()

def train_model(model,input_ids,input_masks,labels,weights,batch_size=64,epochs_frozen=2,epochs_unfrozen=30,verbose=1):
   
    if epochs_frozen:
        
        for layer in model.layers[:3]:
            layer.trainable = False

        with tpu_strategy.scope():
          model.compile(loss={"output":SparseCategoricalFocalLoss(gamma=2)},
                          optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
                          metrics="sparse_categorical_accuracy")

          model.fit([input_ids, input_masks],labels,batch_size=64,epochs=epochs_frozen,verbose=verbose,
                   validation_data=([valid_input_ids,valid_input_masks],valid_labels), class_weight=weights)
        
    if epochs_unfrozen:
        
        for layer in model.layers[:3]:
            layer.trainable = True

        with tpu_strategy.scope():
          model.compile(loss={"output":SparseCategoricalFocalLoss(gamma=2)},
                          optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
                          metrics="sparse_categorical_accuracy")
          model.fit([input_ids, input_masks],labels,batch_size=64,epochs=epochs_unfrozen,verbose=verbose,
                    validation_data=([valid_input_ids,valid_input_masks],valid_labels), class_weight=weights)

train_model(model,train_input_ids,train_input_masks,train_labels,weights)

Downloading:   0%|          | 0.00/757 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/621M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at UBC-NLP/MARBERTv2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/2
50/50 [==============================] - 80s 826ms/step - loss: 1.8038 - sparse_categorical_accuracy: 0.2574 - val_loss: 0.8614 - val_sparse_categorical_accuracy: 0.5268
Epoch 2/2
50/50 [==============================] - 6s 129ms/step - loss: 1.5110 - sparse_categorical_accuracy: 0.4189 - val_loss: 1.4689 - val_sparse_categorical_accuracy: 0.2685
Epoch 1/30


50/50 [==============================] - 200s 2s/step - loss: 1.0815 - sparse_categorical_accuracy: 0.4650 - val_loss: 0.6174 - val_sparse_categorical_accuracy: 0.5819
Epoch 2/30
50/50 [==============================] - 15s 296ms/step - loss: 0.5437 - sparse_categorical_accuracy: 0.6915 - val_loss: 0.4623 - val_sparse_categorical_accuracy: 0.6945
Epoch 3/30
50/50 [==============================] - 15s 298ms/step - loss: 0.1871 - sparse_categorical_accuracy: 0.7877 - val_loss: 0.4861 - val_sparse_categorical_accuracy: 0.7063
Epoch 4/30
50/50 [==============================] - 15s 297ms/step - loss: 0.0950 - sparse_categorical_accuracy: 0.8726 - val_loss: 0.6166 - val_sparse_categorical_accuracy: 0.6472
Epoch 5/30
50/50 [==============================] - 15s 297ms/step - loss: 0.0858 - sparse_categorical_accuracy: 0.8905 - val_loss: 0.5882 - val_sparse_categorical_accuracy: 0.6157
Epoch 6/30
50/50 [==============================] - 15s 298ms/step - loss: 0.0463 - sparse_categorical_accur

In [ ]:
def evaluate_model(model,input_ids,input_masks,labels_OFF,labels_HS,labels_HS_bn):
    predictions=model.predict([input_ids,input_masks])
    print(f"TaskA:{eval_taskA(predictions,labels_OFF)}")
    print(f"TaskB:{eval_taskB(predictions,labels_HS_bn)}")
    print(f"TaskC:{eval_taskC(predictions,labels_HS)}")
evaluate_model(model,valid_input_ids,valid_input_masks,valid_OFF,valid_HS,valid_HS_bn)

TaskA:{'F1_macro': 0.656619045127078, 'Accuracy': 0.658267716535433, 'Precision_macro': 0.7296149360894519, 'Recall_macro': 0.7098720922941877}
TaskB:{'F1_macro': 0.7328774750178371, 'Accuracy': 0.8795275590551181, 'Precision_macro': 0.846822179551004, 'Recall_macro': 0.69}
TaskC:{'F1_macro': 0.40121982436124587, 'Accuracy': 0.8692913385826772, 'Precision_macro': 0.5831299007378742, 'Recall_macro': 0.33775100278244935}


In [ ]:
evaluate_model(model,train_input_ids,train_input_masks,train_OFF,train_HS,train_OFF_not_HS,train_HS_bn)

NameError: ignored

In [ ]:
def zip_file(task):
    with ZipFile(task+".zip", 'w') as myzip:
        myzip.write(task+".txt")
    myzip.close()
    
def submission_gen_taskA(predictions):
    sub=[]
    for i in predictions:
        if i==1:
            sub.append("OFF")
        else :
            sub.append("NOT_OFF")
    pd.DataFrame(sub).to_csv("TaskA.txt",index=False,header=False)

def submission_gen_taskB(predictions):
    sub=[]
    for i in predictions:
        if i==1:
            sub.append("HS")
        else :
            sub.append("NOT_HS")
    pd.DataFrame(sub).to_csv("TaskB.txt",index=False,header=False)

def submission_gen_taskC(predictions):
    sub=[]
    for i in predictions:
        if i==0:
            sub.append("NOT_HS")
        else:
            sub.append("HS"+str(i))
    pd.DataFrame(sub).to_csv("TaskC.txt",index=False,header=False)
    
def submission_gen():
    submission_gen_taskA(eval_taskA(predictions_OFF,labels_OFF,return_predictions=True))
    submission_gen_taskB(eval_taskB2(predictions_HS_bn,labels_HS_bn,return_predictions=True))
    submission_gen_taskC(eval_taskC(predictions_HS,labels_HS,return_predictions=True))

#submission_gen()